In [19]:
import json
import mysql.connector

In [20]:
# Products Insertion

# Read the JSON file
with open("./Data/products.json", encoding="utf-8" ) as file:
  products_json = json.load(file)
  
# Connection to the MySQL database
connection = mysql.connector.connect(
  host = '127.0.0.1',
  user = 'root',
  password = 'password',
  database = 'esuper'
)

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# create the insertion query
insert_products_query = "INSERT INTO product (product_name, category, subcategory, img) VALUES (%s, %s, %s, %s)"

# Iterate through the JSON data and insert it into the table
for item in products_json: 
  product_data_tuple = (item['name'], item["category"], item["subcategory"], item["image"])
  cursor.execute(insert_products_query, product_data_tuple)
  
# Commit the changes to the database
connection.commit()

In [21]:
# Stores Insertion

def get_field(data, field_names):
  for name in field_names:
    if name in data:
      return data[name]
  return "Supermarket"

def flatten_geometry(geometry):
  return geometry.get("coordinates", "")


# Read the JSON file
with open("./Data/stores.json", encoding="utf-8" ) as file:
  stores_json = json.load(file)

# create the insertion query
insert_stores_query = "INSERT INTO store (store_name, longitude, latitude) VALUES (%s, %s, %s)"

for item in stores_json:
  name_or_brand = get_field(item["properties"], ["name", "brand"])
  coordinates = item["geometry"].get("coordinates", "")
  latitude = coordinates[0]
  longitude = coordinates[1]
  store_data_tuple = (name_or_brand, longitude, latitude)
  cursor.execute(insert_stores_query, store_data_tuple)
  
# Commit the changes to the database
connection.commit()

In [22]:
# Categories Insertion

# Read the JSON file
with open("./Data/categories.json", encoding="utf-8" ) as file:
  categories_json = json.load(file)
  
# create the insertion query
insert_category_query = "INSERT INTO category (category_id, category_name) VALUES (%s, %s)"

# Iterate through the JSON data and insert it into the table
for item in categories_json:
  category_data_tuple = (item["id"], item["name"])
  cursor.execute(insert_category_query, category_data_tuple)
  
# Commit the changes to the database
connection.commit()

8016e637b54241f8ad242ed1699bf2da Βρεφικά Είδη
d41744460283406a86f8e4bd5010a66d Καθαριότητα
a8ac6be68b53443bbd93b229e2f9cd34 Ποτά - Αναψυκτικά
8e8117f7d9d64cf1a931a351eb15bd69 Προσωπική φροντίδα
ee0022e7b1b34eb2b834ea334cda52e7 Τρόφιμα
e4b4de2e31fc43b7b68a0fe4fbfad2e6 Αντισηπτικά
2d5f74de114747fd824ca8a6a9d687fa Προστασία Υγείας
662418cbd02e435280148dbb8892782a Για κατοικίδια


In [30]:
# Subcategories Insertion

# Read the JSON file
with open("./Data/categories.json", encoding="utf-8" ) as file:
  subcategories_json = json.load(file)
  
# create the insertion query
insert_subcategory_query = "INSERT INTO subcategory (subcategory_id, subcategory_name, parent_id) VALUES (%s, %s, %s)"

def flatten_subcategories(subcategories):
  return subcategories.get("name"), subcategories.get("uuid")

# Iterate through the JSON data and insert it into the table
for item in subcategories_json:
  parent_category = item["id"]
  subcategories_array = item["subcategories"]
  for subcategory in subcategories_array:
    subcategory_data_tuple = (subcategory["uuid"], subcategory["name"], parent_category)
    cursor.execute(insert_subcategory_query, subcategory_data_tuple)
     
# Commit the changes to the database
connection.commit()

In [31]:
# Close the cursor and connection
cursor.close()
connection.close()